In [1]:
#Import necessary libraries 
import csv
import sqlite3 as sql
import pandas as pd

In [2]:
#The file path of our new database
file = 'C:\\Sqllite\\SouthAfricaCrime.db'

In [4]:
#Creating our database + a message to confirm if we're successfull
try:
  conn = sql.connect(file)
  print("Crime database formed.")
except:
  print("Crime database not formed.")

Crime database formed.


In [10]:
#A function to connect to our database
def get_database_connection():
    con = sql.connect(file)
    return con

In [6]:
#Create first table - Population
pop_data = pd.read_csv('archive/ProvincePopulation.csv')
# Write the data to a sqlite table
pop_data.to_sql('Population', conn, if_exists='replace', index=False)

9

In [8]:
#Checking if our table is not empty
tables = '''SELECT
    *
FROM Population;'''

curs = conn.cursor()
curs.execute(tables)
results = curs.fetchall()

curs.close()
conn.close()

[('Gauteng', 12272263, 18178, 675.1),
 ('Kwazulu/Natal', 10267300, 94361, 108.8),
 ('Mpumalanga', 4039939, 76495, 52.8),
 ('Western Cape', 5822734, 129462, 45.0),
 ('Limpopo', 5404868, 125755, 43.0),
 ('Eastern Cape', 6562053, 168966, 38.8),
 ('North West', 3509953, 104882, 33.5),
 ('Free State', 2745590, 129825, 21.1),
 ('Northern Cape', 1145861, 372889, 3.1)]

In [11]:
#Checking the data types of our columns
conn = get_database_connection()
table_info = "PRAGMA table_info(Population)"
curs = conn.cursor()
for row in curs.execute(table_info):
    print(row)

curs.close()
conn.close()

(0, 'Province', 'TEXT', 0, None, 0)
(1, 'Population', 'INTEGER', 0, None, 0)
(2, 'Area', 'INTEGER', 0, None, 0)
(3, 'Density', 'REAL', 0, None, 0)


In [27]:
conn = get_database_connection()
#Create second table - SouthAfricaCrimeStats
pop_data = pd.read_csv('archive/SouthAfricaCrimeStats_v2_Update.csv')
# Write the data to a sqlite table
pop_data.to_sql('SouthAfricaCrime', conn, if_exists='replace', index=False)

339471

In [28]:
conn = get_database_connection()
table = '''SELECT
    *
FROM SouthAfricaCrime;'''

curs = conn.cursor()
curs.execute(tables)
results = curs.fetchall()

Crime_pd = pd.read_sql_query(table, conn)

curs.close()
conn.close()

Crime_pd.head(10)

,Province,Station,Category,Year,Reports
0,Western Cape,Cape Town Central,All theft not mentioned elsewhere,2005-2006,6692
1,Western Cape,Cape Town Central,All theft not mentioned elsewhere,2006-2007,6341
2,Western Cape,Cape Town Central,All theft not mentioned elsewhere,2007-2008,5966
3,Western Cape,Cape Town Central,All theft not mentioned elsewhere,2008-2009,5187
4,Western Cape,Cape Town Central,All theft not mentioned elsewhere,2009-2010,4985
5,Western Cape,Cape Town Central,All theft not mentioned elsewhere,2010-2011,5127
6,Western Cape,Cape Town Central,All theft not mentioned elsewhere,2011-2012,5285
7,Western Cape,Cape Town Central,All theft not mentioned elsewhere,2012-2013,5937
8,Western Cape,Cape Town Central,All theft not mentioned elsewhere,2013-2014,5600
9,Western Cape,Cape Town Central,All theft not mentioned elsewhere,2014-2015,5335


In [29]:
#First observations
#The yearly columns are not useful. We'll have to first change the columns names then unpivot the columns to have years in rows
#for better futur analysis.

In [30]:
#Let's analyze the crime table
#First check if there're any missing values
Crime_pd.apply(lambda x: sum(x.isnull()))

Province    0
Station     0
Category    0
Year        0
Reports     0
dtype: int64

In [35]:
conn = get_database_connection()

table = '''SELECT Year, COUNT(*)
FROM SouthAfricaCrime
GROUP BY Year
ORDER BY COUNT(*) DESC;'''

Year_pd = pd.read_sql_query(table, conn)

Year_pd

,Year,COUNT(*)
0,2015-2016,30861
1,2014-2015,30861
2,2013-2014,30861
3,2012-2013,30861
4,2011-2012,30861
5,2010-2011,30861
6,2009-2010,30861
7,2008-2009,30861
8,2007-2008,30861
9,2006-2007,30861


In [17]:
#We know there no empty values i.e., less work to do
#Now let's analyze the column distribution of the text columns
conn = get_database_connection()

table = '''SELECT Province, COUNT(*)
FROM SouthAfricaCrime
GROUP BY Province
ORDER BY COUNT(*) DESC;'''

Province_pd = pd.read_sql_query(table, conn)

Province_pd

,Province,COUNT(*)
0,Eastern Cape,5292
1,Kwazulu/Natal,5022
2,Western Cape,4050
3,Gauteng,3861
4,Free State,2970
5,Limpopo,2673
6,Northern Cape,2457
7,Mpumalanga,2322
8,North West,2214


In [31]:
conn = get_database_connection()

table = '''SELECT Category, COUNT(*)
FROM SouthAfricaCrime
GROUP BY Category
ORDER BY COUNT(*) DESC;'''

Category_pd = pd.read_sql_query(table, conn)

Category_pd

,Category,COUNT(*)
0,Truck hijacking,12573
1,Theft out of or from motor vehicle,12573
2,Theft of motor vehicle and motorcycle,12573
3,Stock-theft,12573
4,Shoplifting,12573
5,Sexual offences as result of police action,12573
6,Sexual Offences,12573
7,Robbery with aggravating circumstances,12573
8,Robbery of cash in transit,12573
9,Robbery at residential premises,12573


In [32]:
#There are 27 categories which from a quick glance could be way less.
#1. We could regroup all theft, robberies into petty theft, grand theft and grand felony theft.
#2. Regroup all sexual offences to sexual assault.
#3. Regroup all assault into common assault and violent assault
#4. Convert Driving under the influence of alcohol or drugs to DUI
#5. Convert Malicious damage to property to Property damage
#6. Regroup all burglaries into Low-level burglary and High-level burglary

In [33]:
#1. We could regroup all theft, robberies into petty theft, grand theft and grand felony theft.
sql_query = '''UPDATE SouthAfricaCrime
SET Category = 
CASE Category
WHEN 'Truck hijacking' THEN 'Grand theft'
WHEN 'Theft out of or from motor vehicle' THEN 'Grand theft'
WHEN 'Theft of motor vehicle and motorcycle' THEN 'Grand theft'
WHEN 'Stock-theft' THEN 'Petty theft'
WHEN 'Shoplifting' THEN 'Petty theft'
WHEN 'Sexual offences as result of police action' THEN 'Sexual assault'
WHEN 'Sexual Offences' THEN 'Sexual assualt'
WHEN 'Robbery with aggravating circumstances' THEN 'Grand felony theft'
WHEN 'Robbery of cash in transit' THEN 'Grand felony theft'
WHEN 'Robbery at residential premises' THEN 'Grand theft'
WHEN 'Robbery at non-residential premises' THEN 'Petty theft'
WHEN 'Common robbery' THEN 'Petty theft'
WHEN 'Carjacking' THEN 'Grand theft'
WHEN 'Bank robbery' THEN 'Grand felony theft'
WHEN 'All theft not mentioned elsewhere' THEN 'Petty theft'
WHEN 'Malicious damage to property' THEN 'Property damage'
WHEN 'Illegal possession of firearms and ammunition' THEN 'Firearm & Ammo possession'
WHEN 'Driving under the influence of alcohol or drugs' THEN 'DUI'
WHEN 'Burglary at residential premises' THEN 'Residential burglary'
WHEN 'Burglary at non-residential premises' THEN 'Non-residential burglary'
WHEN 'Assault with the intent to inflict grievous bodily harm' THEN 'Violent assault'
ELSE Category
END
WHERE Category IS NOT NULL;'''

conn.execute(sql_query)
conn.commit()
conn.close()

In [34]:
#Transform the rest
conn = get_database_connection()
sql_query = '''UPDATE SouthAfricaCrime
SET Category = 
CASE Category
WHEN 'Sexual assualt' THEN 'Sexual assault'
WHEN 'Malicious damage to property' THEN 'Property damage'
WHEN 'Illegal possession of firearms and ammunition' THEN 'Firearm & Ammo possession'
WHEN 'Driving under the influence of alcohol or drugs' THEN 'DUI'
WHEN 'Burglary at residential premises' THEN 'Residential burglary'
WHEN 'Burglary at non-residential premises' THEN 'Non-residential burglary'
WHEN 'Assault with the intent to inflict grievous bodily harm' THEN 'Violent assault'
ELSE Category
END
WHERE Category IS NOT NULL;'''

conn.execute(sql_query)
conn.commit()
conn.close()

In [36]:
#Transform the Year column into proper columns
conn = get_database_connection()
sql_query = '''UPDATE SouthAfricaCrime
SET Year = 
CASE Year
WHEN '2005-2006' THEN '2005'
WHEN '2006-2007' THEN '2006'
WHEN '2007-2008' THEN '2007'
WHEN '2008-2009' THEN '2008'
WHEN '2009-2010' THEN '2009'
WHEN '2010-2011' THEN '2010'
WHEN '2011-2012' THEN '2011'
WHEN '2012-2013' THEN '2012'
WHEN '2013-2014' THEN '2013'
WHEN '2014-2015' THEN '2014'
WHEN '2015-2016' THEN '2015'
ELSE Year
END
WHERE Year IS NOT NULL;'''

conn.execute(sql_query)
conn.commit()
conn.close()

In [19]:
table = '''SELECT Station, COUNT(*)
FROM SouthAfricaCrime
GROUP BY Station
ORDER BY COUNT(*) DESC;'''

Station_pd = pd.read_sql_query(table, conn)

Station_pd

,Station,COUNT(*)
0,Zwelitsha,27
1,Zonkizizwe,27
2,Zele,27
3,Zeerust,27
4,Zebediela,27
...,...,...
1138,'Simon''S Town',27
1139,'Rankin''S Pass',27
1140,'Pilgrim''S Rest',27
1141,'Low''S Creek',27


In [37]:
conn = get_database_connection()
table = '''SELECT
    *
FROM SouthAfricaCrime;'''

curs = conn.cursor()
curs.execute(table)
results = curs.fetchall()

Crime_pd = pd.read_sql_query(table, conn)

curs.close()
conn.close()

Crime_pd.head(10)

,Province,Station,Category,Year,Reports
0,Western Cape,Cape Town Central,Petty theft,2005,6692
1,Western Cape,Cape Town Central,Petty theft,2006,6341
2,Western Cape,Cape Town Central,Petty theft,2007,5966
3,Western Cape,Cape Town Central,Petty theft,2008,5187
4,Western Cape,Cape Town Central,Petty theft,2009,4985
5,Western Cape,Cape Town Central,Petty theft,2010,5127
6,Western Cape,Cape Town Central,Petty theft,2011,5285
7,Western Cape,Cape Town Central,Petty theft,2012,5937
8,Western Cape,Cape Town Central,Petty theft,2013,5600
9,Western Cape,Cape Town Central,Petty theft,2014,5335


In [38]:
Crime_pd.tail(10)

,Province,Station,Category,Year,Reports
339461,Western Cape,Lentegeur,Drug-related crime,2006,0
339462,Western Cape,Lentegeur,Drug-related crime,2007,0
339463,Western Cape,Lentegeur,Drug-related crime,2008,0
339464,Western Cape,Lentegeur,Drug-related crime,2009,0
339465,Western Cape,Lentegeur,Drug-related crime,2010,0
339466,Western Cape,Lentegeur,Drug-related crime,2011,0
339467,Western Cape,Lentegeur,Drug-related crime,2012,0
339468,Western Cape,Lentegeur,Drug-related crime,2013,1126
339469,Western Cape,Lentegeur,Drug-related crime,2014,1933
339470,Western Cape,Lentegeur,Drug-related crime,2015,2050


In [43]:
#Unpivoting the data made it easier to remove the zero reported crimes from our dataset
#I noticed some zero values in the reports column which escaped the empty value check
#We have to remove these rows as they do not have any value for our analysis
conn = get_database_connection()
table = '''SELECT
    COUNT(*)
FROM SouthAfricaCrime;'''

curs = conn.cursor()
curs.execute(table)
results = curs.fetchall()

print("Number of rows before removal: " + str(results))

sql_query = '''DELETE FROM SouthAfricaCrime WHERE Reports = 0;'''
conn.execute(sql_query)
conn.commit()

curs.execute(table)
results = curs.fetchall()
print("Number of rows after removal: " + str(results))

curs.close()
conn.close()

Number of rows before removal: [(339471,)]
Number of rows after removal: [(255414,)]


In [44]:
#25% of data removed

In [48]:
#Now let's begin data exploration
#It'll be interesting to know which years had the most crime
conn = get_database_connection()

table = '''SELECT SUM(Reports) Crimes, Year
FROM SouthAfricaCrime
GROUP BY Year
ORDER BY Crimes DESC;'''

ReportYear_pd = pd.read_sql_query(table, conn)

ReportYear_pd


,Crimes,Year
0,2206506,2014
1,2204000,2013
2,2182999,2015
3,2176557,2005
4,2151032,2012
5,2145388,2009
6,2138717,2006
7,2121884,2008
8,2106560,2011
9,2091348,2010


In [49]:
#2014 had the most reports in contrast 2007 had the least reports. But there is little deviation between the years.
#It is between the range of 2M and 2.2M reports

In [51]:
#Let's check the most recurrent crime category
conn = get_database_connection()

table = '''SELECT SUM(Reports) As Crimes, Category
FROM SouthAfricaCrime
GROUP BY Category
ORDER BY Crimes DESC;'''

CategoryReport_pd = pd.read_sql_query(table, conn)

CategoryReport_pd

,Crimes,Category
0,6010435,Petty theft
1,2763950,Residential burglary
2,2502303,Grand theft
3,2179207,Violent assault
4,2043267,Common assault
5,1879871,Drug-related crime
6,1397845,Property damage
7,1288428,Grand felony theft
8,807206,Commercial crime
9,751717,Non-residential burglary


In [52]:
#Petty theft is the most recurrent crime by a huge margin while arson is the least recurrent crime and it's also an outlier

In [152]:
table = '''
SELECT Province, 
       SUM(Reports) CrimeReports, 
       SUM(Reports)*100/(SELECT SUM(Reports) FROM SouthAfricaCrime) Report_Percent
FROM SouthAfricaCrime
GROUP BY Province
ORDER BY CrimeReports DESC;'''

ProvinceReport_pd = pd.read_sql_query(table, conn)

ProvinceReport_pd

,Province,CrimeReports,Report_Percent
0,Gauteng,6855654,29
1,Western Cape,4792127,20
2,Kwazulu/Natal,3808898,16
3,Eastern Cape,2370079,10
4,Free State,1397044,5
5,Mpumalanga,1370933,5
6,North West,1255258,5
7,Limpopo,1201185,5
8,Northern Cape,546262,2


In [64]:
table = '''SELECT SUM(Reports) As Crimes, Province, Year
FROM SouthAfricaCrime
GROUP BY Province, Year
ORDER BY Year DESC;'''

ProvinceReportYear_pd = pd.read_sql_query(table, conn)

ProvinceReportYear_pd

,Crimes,Province,Year
0,196089,Eastern Cape,2015
1,117688,Free State,2015
2,622218,Gauteng,2015
3,342772,Kwazulu/Natal,2015
4,129323,Limpopo,2015
...,...,...,...
94,106983,Limpopo,2005
95,134829,Mpumalanga,2005
96,118840,North West,2005
97,56515,Northern Cape,2005


In [54]:
table = '''SELECT *
FROM Population;'''

Province_pd = pd.read_sql_query(table, conn)

Province_pd

,Province,Population,Area,Density
0,Gauteng,12272263,18178,675.1
1,Kwazulu/Natal,10267300,94361,108.8
2,Mpumalanga,4039939,76495,52.8
3,Western Cape,5822734,129462,45.0
4,Limpopo,5404868,125755,43.0
5,Eastern Cape,6562053,168966,38.8
6,North West,3509953,104882,33.5
7,Free State,2745590,129825,21.1
8,Northern Cape,1145861,372889,3.1


In [153]:
#We want to know the crime rate by province.
#The reason to analyze 2015 is because, the population data is the most likely recent by nature. 
#Population number rarely stays stagnant.
#Therefore we cannot assume the populatio number of each province in 2005 is the one in 2015.
table = '''
WITH sac_2015 AS
   (
    SELECT Province, Reports, Station, Year
    FROM SouthAfricaCrime 
    WHERE Year = 2015
   ) 
SELECT  sac.Province, 
        SUM(sac.Reports) CrimesReported, 
        p.Population, 
        p.Density PopulationDensity,
        COUNT(sac.Station) PoliceStation,
        CAST(SUM(sac.Reports) AS FLOAT) / (p.Population) CrimeRate
FROM sac_2015 sac 
INNER JOIN Population p ON p.Province = sac.Province
GROUP BY sac.Province
ORDER BY CrimeRate DESC;'''

ProvinceReport_pd = pd.read_sql_query(table, conn)

ProvinceReport_pd

,Province,CrimesReported,Population,PopulationDensity,PoliceStation,CrimeRate
0,Western Cape,490383,5822734,45.0,3242,0.084219
1,Gauteng,622218,12272263,675.1,3398,0.050701
2,Northern Cape,50665,1145861,3.1,1589,0.044216
3,Free State,117688,2745590,21.1,2183,0.042864
4,Kwazulu/Natal,342772,10267300,108.8,4073,0.033385
5,North West,114335,3509953,33.5,1732,0.032575
6,Eastern Cape,196089,6562053,38.8,3823,0.029882
7,Mpumalanga,119526,4039939,52.8,1891,0.029586
8,Limpopo,129323,5404868,43.0,2115,0.023927


In [ ]:
#Western Cape is the most dangerous Province in 2015 with quite a significant margin over other provinces,
#Gauteng has slightly more reported crimes but has the double the population of Western Cape
#and Limpopo is the least crime infested province in 2015

In [139]:
#Now let's take a deep dive into Western Cape to see which crimes are the most reported
conn = get_database_connection()
table = '''
SELECT SUM(Reports) Crimes, Category
FROM SouthAfricaCrime 
WHERE Year = 2015 AND Province = "Western Cape"
GROUP BY Category
ORDER BY Crimes DESC
;'''

WestCapeCategoryReport_pd = pd.read_sql_query(table, conn)

WestCapeCategoryReport_pd

,Crimes,Category
0,120843,Petty theft
1,93996,Drug-related crime
2,54488,Grand theft
3,47569,Residential burglary
4,41304,Common assault
5,29217,Property damage
6,25539,Violent assault
7,23756,Grand felony theft
8,13344,Non-residential burglary
9,11725,DUI


In [141]:
#Let's also take a deeper dive into Western Cape to see which police station has the most reported crimes
table = '''
SELECT Station,SUM(Reports) Crimes
FROM SouthAfricaCrime 
WHERE Year = 2015 AND Province = "Western Cape"
GROUP BY Station
ORDER BY Crimes DESC
LIMIT 10
;'''

WestCapeStationReport_pd = pd.read_sql_query(table, conn)

WestCapeStationReport_pd

,Station,Crimes
0,Mitchells Plain,19499
1,Cape Town Central,17785
2,Kraaifontein,11939
3,Worcester,10875
4,Nyanga,10785
5,Stellenbosch,10141
6,Bellville,9508
7,Delft,9452
8,Milnerton,8471
9,Bishop Lavis,8397


In [88]:
#let's take a deep dive into Gauteng to see which crimes are the most reported
table = '''
SELECT SUM(Reports) Crimes, Category
FROM SouthAfricaCrime 
WHERE Year = 2015 AND Province = "Gauteng"
GROUP BY Category
ORDER BY Crimes DESC
;'''

GautengCategoryReport_pd = pd.read_sql_query(table, conn)

GautengCategoryReport_pd

,Crimes,Category
0,150535,Petty theft
1,84715,Grand theft
2,64968,Residential burglary
3,55442,Drug-related crime
4,49826,Grand felony theft
5,44255,Common assault
6,42790,Violent assault
7,34320,Property damage
8,32287,DUI
9,23680,Commercial crime


In [89]:
#let's take a deep dive into Northern Cape to see which crimes are the most reported
table = '''
SELECT SUM(Reports) Crimes, Category
FROM SouthAfricaCrime 
WHERE Year = 2015 AND Province = "Northern Cape"
GROUP BY Category
ORDER BY Crimes DESC
;'''

NorthCapeCategoryReport_pd = pd.read_sql_query(table, conn)

NorthCapeCategoryReport_pd

,Crimes,Category
0,11530,Petty theft
1,8889,Violent assault
2,6469,Residential burglary
3,4598,Common assault
4,4344,Drug-related crime
5,3363,Grand theft
6,3084,Property damage
7,2203,Non-residential burglary
8,1741,Sexual assault
9,1388,Grand felony theft


In [90]:
#Key Takeaways from the three highest rated crime provinces
#As expected Petty theft is the most common crime. Other highly reported crimes are Residential burglaries, Common assault
#and drug related crimes
#The least reported crimes are Arson, Murder, Attempted murder, and Firearm possesion
#Western Cape has a really high number of reported drug crimes, double that of Gauteng(which is double in population size)

In [ ]:
#We know want to analyze crime reporting over the decade. We would take the difference between crimes reported in 2005 
#(First year of reporting) and 2015 (most recent year of reporting)

In [123]:
table = '''
    WITH sac_2015 AS
    (
        SELECT Province, Reports, Category
        FROM SouthAfricaCrime 
        WHERE Year = 2015
    )
    
    SELECT Category, SUM(Reports) Crimes_2015
    FROM sac_2015
    GROUP BY 
        Category
    ORDER BY 
        Category DESC;
    '''

Reports_2015_pd = pd.read_sql_query(table, conn)

Reports_2015_pd

,Category,Crimes_2015
0,Violent assault,182933
1,Sexual assault,57725
2,Residential burglary,250606
3,Property damage,119901
4,Petty theft,507681
5,Non-residential burglary,75008
6,Murder,18673
7,Grand theft,229801
8,Grand felony theft,132670
9,Firearm & Ammo possession,14772


In [124]:
table = '''
    WITH sac_2005 AS
    (
        SELECT Province, Reports, Category
        FROM SouthAfricaCrime 
        WHERE Year = 2005
    )
    
    SELECT Category, SUM(Reports) Crimes_2005
    FROM sac_2005
    GROUP BY 
        Category
    ORDER BY 
        Category DESC;
    '''

Reports_2005_pd = pd.read_sql_query(table, conn)

Reports_2005_pd

,Category,Crimes_2005
0,Violent assault,225659
1,Sexual assault,67064
2,Residential burglary,261403
3,Property damage,141776
4,Petty theft,594254
5,Non-residential burglary,54217
6,Murder,18455
7,Grand theft,247966
8,Grand felony theft,119684
9,Firearm & Ammo possession,13239


In [133]:
Reports_diff_pd = Reports_2005_pd[['Category']].copy()
Reports_diff_pd['Diff%'] = ((Reports_2015_pd.Crimes_2015 - Reports_2005_pd.Crimes_2005)/Reports_2015_pd.Crimes_2015)*100
Reports_diff_pd = Reports_diff_pd.sort_values(by = 'Diff%', ascending = False)

In [134]:
Reports_diff_pd

,Category,Diff%
10,Drug-related crime,63.420601
11,DUI,56.569808
5,Non-residential burglary,27.718377
13,Commercial crime,25.753393
9,Firearm & Ammo possession,10.377742
8,Grand felony theft,9.788196
6,Murder,1.167461
2,Residential burglary,-4.308357
7,Grand theft,-7.904665
14,Attempted murder,-12.368290


In [ ]:
#Drug related crimes and DUI reporting have he highest increases by 55+% while arson has decreased by 43%
#It is important to note, this is only crime reporting and it is widely know some crimes such as sexual assault are very
#under-reported due to societal failures 

In [135]:
#We want to know how crime category reporting evolved over the decade
#table = '''
#SELECT  s2.Category, 
#        COALESCE(s2.Reports,0) - COALESCE(s1.Reports,0) CrimeDiff,
#        s1.Category,
#        s2.Category
#FROM SouthAfricaCrime sac
#LEFT JOIN (SELECT Province, Reports, Category
#    FROM SouthAfricaCrime 
#    WHERE Year = 2015) s1 on s1.Category = sac.Category
#LEFT JOIN (SELECT Province, Reports, Category
#    FROM SouthAfricaCrime 
#    WHERE Year = 2005) s2 on s2.Category = sac.Category
#WHERE s1.Category is not null or s2.Category is not null;
#'''

#CategoryDiff_pd = pd.read_sql_query(table, conn)

#CategoryDiff_pd

In [137]:
conn.close()

In [142]:
conn = get_database_connection()

table = '''SELECT Category, COUNT(*)
FROM SouthAfricaCrime
GROUP BY Category
ORDER BY COUNT(*) DESC;'''

Category_pd = pd.read_sql_query(table, conn)

Category_pd

,Category,COUNT(*)
0,Petty theft,50921
1,Grand theft,40738
2,Sexual assault,13807
3,Grand felony theft,13397
4,Violent assault,12324
5,Residential burglary,12286
6,Common assault,12246
7,Property damage,12188
8,Drug-related crime,12127
9,Non-residential burglary,11772
